#### Is important to have "fresh" data in "puntos_de_control"

In [7]:
# Imports and Settings
import pickle

from osgeo import gdal, ogr

from landsat8 import write_geotiff

from utils import (
    print_cm, test_class_name_to_train_label, train_label_to_test_class_name, 
    extract_test_mask, add_label_from_reference_name,
    CLASS_NAME_TO_INT, INT_TO_CLASS_NAME)


verification_vector_data_path = "real_data/puntos_control/CLASIFICACION_Y_ROI.shp"

# We need projection and GeoTransform
data_date = "150201"
raster_data_path = "real_data/%s/img/L8_229_82_%s.tif" % (data_date, data_date)
print("Reading the input: %s" % raster_data_path)
try:
    raster_dataset = gdal.Open(raster_data_path, gdal.GA_ReadOnly)
except RuntimeError as e:
    report_and_exit(str(e))
geo_transform = raster_dataset.GetGeoTransform()
proj = raster_dataset.GetProjectionRef()
# ##################################

Reading the input: real_data/150201/img/L8_229_82_150201.tif


In [8]:
# Esto se hace una sola vez
add_label_from_reference_name(verification_vector_data_path)

In [9]:

rows, cols = raster_dataset.GetRasterBand(1).ReadAsArray().shape
test_ds = extract_test_mask(verification_vector_data_path, rows, cols, geo_transform, proj, 'reference')

In [10]:
test_band = test_ds.GetRasterBand(1)
labeled_pixels = test_band.ReadAsArray()

In [11]:
output_fname = "verification_by_class.tiff"
write_geotiff(output_fname, labeled_pixels, geo_transform, proj)
print("Verification image created: %s" % output_fname)

Verification image created: verification_by_class.tiff


In [12]:
with open('mask_lote_by_class.pickle', 'wb') as fout:
    pickle.dump(labeled_pixels, fout)